In [3]:
from transformer_class import SimilarityModel
from data_utils import PhraseDataset
from torch.utils.data import DataLoader
import torch

# Set up DataLoaders

In [5]:
train_loader = DataLoader(PhraseDataset('processed/train.csv'), batch_size=32, shuffle=True)
val_loader = DataLoader(PhraseDataset('processed/valid.csv'), batch_size=32, shuffle=False)
test_loader = DataLoader(PhraseDataset('processed/test.csv'), batch_size=32, shuffle=False)

# Set up Model, Loss function and Optimizer

In [7]:
bert_pretrained = 'bert-base-uncased'

model = SimilarityModel(bert_pretrained)
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

KeyboardInterrupt: 

# Train Model

In [ ]:
from transformer_class import train
train(model, train_loader, val_loader, device, optimizer, loss, 3)

# Evaluate Model